In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

ModuleNotFoundError: No module named 'keras'

In [3]:
import pandas as pd

In [5]:
import csv

file_path1 = 'USvideos.csv'
file_path2 = 'GBvideos.csv'

# Open the CSV file 1
with open(file_path1, 'r', encoding='utf-8', errors='ignore') as file:
    try:
        # Create a CSV reader
        reader = csv.reader(file)
        
        # Read the CSV data
        rows = [row for row in reader]
    except csv.Error as e:
        sys.exit(f'Error while reading CSV file: {str(e)}')

# Convert the rows to a DataFrame
USdataset = pd.DataFrame(rows)

# Open the CSV file 2
with open(file_path2, 'r', encoding='utf-8', errors='ignore') as file:
    try:
        # Create a CSV reader
        reader = csv.reader(file)
        
        # Read the CSV data
        rows = [row for row in reader]
    except csv.Error as e:
        sys.exit(f'Error while reading CSV file: {str(e)}')

# Convert the rows to a DataFrame
GBdataset = pd.DataFrame(rows)

In [6]:
# GBdataset = GBdataset.drop(GBdataset.index[0])

# Concatenate the two dataframes with new index
# dataset = pd.concat([USdataset, GBdataset], ignore_index=True)
dataset = USdataset

dataset=dataset.drop([11,12,13,14,15,16,17,18,19,20],axis=1)

header = dataset.loc[0].to_list()

dataset = dataset.drop(dataset.index[0])

dataset.columns = header

In [7]:
dataset.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
1,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
2,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
3,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
4,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
5,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [8]:
dataset = dataset.drop(['views','likes','dislikes','comment_total','thumbnail_link','date'], axis=1)

In [9]:
dataset['video_id'].nunique()

2364

In [10]:
newDataset = dataset.drop_duplicates(subset=['video_id'], ignore_index=True)

In [11]:
newDataset

,video_id,title,channel_title,category_id,tags
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none]
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...
...,...,...,...,...,...
2359,BpGrq3LgbfA,Josie and The Pussycats ft. Cheryl Blossom | M...,BlackStardust,1,Riverdale - Josie and The Pussycats - CW - Che...
2360,S9VIKOuZcds,My Sweet Jax (Tribute to a Cat),Hot Dad,23,cat|feline|pets|beloved|family member|grief|gr...
2361,c74FjEqhCsc,Pistons LIVE Postgame 10.18.17: Stan Van Gundy,FOX SportsDetroit,17,[none]
2362,a5NIg5yyHWo,Pawn Stars: An Original 1978 Superman Costume ...,HISTORY,24,history|history channel|history shows|history ...


In [12]:
# Drom row if length of tag is less than 10
newDataset = newDataset[newDataset['tags'].str.len() > 10]

In [13]:
newDataset

,video_id,title,channel_title,category_id,tags
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...
5,cMKX2tE5Luk,The Disaster Artist | Official Trailer HD | A24,A24,1,a24|a24 films|a24 trailers|independent films|t...
...,...,...,...,...,...
2358,7_GaeAoLMWY,Keyshia Cole Performs Incapable,Wendy Williams,24,Keyshia Cole|wendy williams|the wendy williams...
2359,BpGrq3LgbfA,Josie and The Pussycats ft. Cheryl Blossom | M...,BlackStardust,1,Riverdale - Josie and The Pussycats - CW - Che...
2360,S9VIKOuZcds,My Sweet Jax (Tribute to a Cat),Hot Dad,23,cat|feline|pets|beloved|family member|grief|gr...
2362,a5NIg5yyHWo,Pawn Stars: An Original 1978 Superman Costume ...,HISTORY,24,history|history channel|history shows|history ...


In [14]:
newDataset['tags'].value_counts()

tags
The Late Show|Stephen Colbert|Colbert|Late Show|celebrities|late night|talk show|skits|bit|monologue|The Late Late Show|Late Late Show|letterman|david letterman|comedian|impressions|CBS|joke|jokes|funny|funny video|funny videos|humor|celebrity|celeb|hollywood|famous|James Corden|Corden|Comedy                                                                                                      13
James Corden|The Late Late Show|Colbert|late night|late night show|Stephen Colbert|Comedy|monologue|comedian|impressions|celebrities|carpool|karaoke|CBS|Late Late Show|Corden|joke|jokes|funny|funny video|funny videos|humor|celebrity|celeb|hollywood|famous                                                                                                                                            12
cupcakes|how to make vanilla cupcakes|over the top recipes|easy cupcake recipes|vanilla cupcakes|chocolate cupcakes|french macarons|how to make macarons|the scran line|the scranline|nick makrides|pas

In [15]:
# newDataset.drop(newDataset[newDataset['tags']=='[none]'].index, inplace=True)

In [16]:
newDataset['tags'] = newDataset['tags'].str.split('|')

C:\Users\nadil\AppData\Local\Temp\ipykernel_17884\3861711897.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDataset['tags'] = newDataset['tags'].str.split('|')


In [17]:
newDataset['category_id'].nunique()

16

In [18]:
newDataset = newDataset.drop(['video_id'], axis=1)

In [19]:
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [73]:
X = newDataset[['title', 'channel_title', 'tags']]  # Features
y = newDataset['category_id']  # Target variable

# Split the data into training and testing sets
test_size = 0.2  # Specify the proportion of the dataset for testing (e.g., 0.2 for 20%)
random_state = 42  # Set a random state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

In [74]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [75]:
# shapes of the training and testing datasets
print(f'Training data shape: {X_train.shape}')
print(f'Testing data shape: {X_test.shape}')
print(f'Training target shape: {y_train.shape}')
print(f'Testing target shape: {y_test.shape}')

Training data shape: (1762, 3)
Testing data shape: (441, 3)
Training target shape: (1762,)
Testing target shape: (441,)


In [76]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [77]:
# max_words = 10000  # Maximum number of words to keep
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(X_train)
# X_train_seq = tokenizer.texts_to_sequences(X_train, oov_token='<UNK>')

# max_sequence_length = 100  # Maximum sequence length
# X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)

# model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
# model.add(LSTM(units=64))  # You can adjust the number of LSTM units
# model.add(Dense(units=32, activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model
# model.fit([X_train_padded], y_train, batch_size=32, epochs=10, validation_split=0.2)

# # Evaluate the model
# # Prepare your test data (X_test_text, X_test_other, y_test) in a similar way as the training data
# X_test_seq = tokenizer.texts_to_sequences(X_test)
# X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length)
# loss, accuracy = model.evaluate([X_test_padded], y_test)
# print("Loss:", loss)
# print("Accuracy:", accuracy)

In [78]:
# max_words = 10000  # Maximum number of words to keep
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(X_train)
# X_train_text_seq = tokenizer.texts_to_sequences(X_train)

# # Pad the text sequences to have the same length
# max_sequence_length = 100  # Maximum sequence length
# X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)

# # Define the models for different input features
# text_model = Sequential()
# text_model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
# text_model.add(LSTM(units=64))

# # Add additional layers for prediction
# predictions = Dense(units=32, activation='softmax')(text_model)

# # # Define the final model
# # model = Model(inputs=[text_model.input, other_model.input], outputs=predictions)

# # Compile and train the model
# text_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# text_model.fit([X_train_padded], y_train, batch_size=32, epochs=10, validation_split=0.2)

In [79]:
# max_words = 10000  # Maximum number of words to keep
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(X_train)
# X_train_seq = tokenizer.texts_to_sequences(X_train)

# # Pad the text sequences to have the same length
# max_sequence_length = 100  # Maximum sequence length
# X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length)

# # Define the text model
# model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
# model.add(LSTM(units=64))
# model.add(Dense(units=32, activation='softmax'))  # Output layer with softmax activation

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Train the model
# model.fit(X_train_padded, y_train, batch_size=32, epochs=10, validation_split=0.2)

# # Evaluate the model
# # Prepare your test data (X_test_text, y_test) in a similar way as the training data
# X_test_text_seq = tokenizer.texts_to_sequences(X_test)
# X_test_text_padded = pad_sequences(X_test_text_seq, maxlen=max_sequence_length)
# loss, accuracy = model.evaluate(X_test_text_padded, y_test)
# print("Loss:", loss)
# print("Accuracy:", accuracy)

In [120]:
X_train_new = pd.DataFrame()
X_test_new = pd.DataFrame()

max_words = 10000  # Maximum number of words to keep
tokenizer1 = Tokenizer(num_words=max_words, oov_token='<UNK>')
tokenizer1.fit_on_texts(X_train['title'])
X_train_new['title']=tokenizer1.texts_to_sequences(X_train['title'])
X_test_new['title']=tokenizer1.texts_to_sequences(X_test['title'])  

tokenizer2 = Tokenizer(num_words=max_words, oov_token='<UNK>')
tokenizer2.fit_on_texts(X_train['channel_title'])
X_train_new['channel_title']=tokenizer2.texts_to_sequences(X_train['channel_title'])
X_test_new['channel_title']=tokenizer2.texts_to_sequences(X_test['channel_title'])

tokenizer3 = Tokenizer(num_words=max_words, oov_token='<UNK>')
tokenizer3.fit_on_texts(X_train['tags'])
X_train_new['tags']=tokenizer3.texts_to_sequences(X_train['tags'])
X_test_new['tags']=tokenizer3.texts_to_sequences(X_test['tags'])

In [121]:
# Pad the text sequences to have the same length
max_sequence_length = 100  # Maximum sequence length
X_train_new_seq = pd.DataFrame()
X_test_new_seq = pd.DataFrame()

titleList = X_train_new['title'].tolist()
titleListTest = X_test_new['title'].tolist()
channel_titleList = X_train_new['channel_title'].tolist()
channel_titleListTest = X_test_new['channel_title'].tolist()
tagsList = X_train_new['tags'].tolist()
tagsListTest = X_test_new['tags'].tolist()

titleList_seq = pad_sequences(titleList, truncating='post', padding='post', maxlen=max_sequence_length)
titleListTest_seq = pad_sequences(titleListTest, truncating='post', padding='post', maxlen=max_sequence_length)
channel_titleList_seq = pad_sequences(channel_titleList, truncating='post', padding='post', maxlen=max_sequence_length)
channel_titleListTest_seq = pad_sequences(channel_titleListTest, truncating='post', padding='post', maxlen=max_sequence_length)
tagsList_seq = pad_sequences(tagsList, truncating='post', padding='post', maxlen=max_sequence_length)
tagsListTest_seq = pad_sequences(tagsListTest, truncating='post', padding='post', maxlen=max_sequence_length)

X_train_new_seq['title'] = titleList_seq.tolist()
X_test_new_seq['title'] = titleListTest_seq.tolist()
X_train_new_seq['channel_title'] = channel_titleList_seq.tolist()
X_test_new_seq['channel_title'] = channel_titleListTest_seq.tolist()
X_train_new_seq['tags'] = tagsList_seq.tolist()
X_test_new_seq['tags'] = tagsListTest_seq.tolist()

In [122]:
X_train_new_seq

,title,channel_title,tags
0,"[499, 1838, 250, 709, 0, 0, 0, 0, 0, 0, 0, 0, ...","[540, 322, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4367, 68, 4368, 4369, 4370, 4371, 4372, 4373,..."
1,"[1073, 20, 710, 60, 48, 26, 49, 83, 0, 0, 0, 0...","[20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[35, 41, 217, 170, 218, 219, 171, 42, 69, 79, ..."
2,"[374, 500, 711, 8, 375, 4, 66, 51, 0, 0, 0, 0,...","[218, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1550, 1093, 1094, 24, 624, 420, 2324, 1095, 4..."
3,"[298, 1074, 1839, 24, 299, 712, 0, 0, 0, 0, 0,...","[113, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[15, 6, 7, 625, 73, 31, 423, 1097, 59, 1555, 2..."
4,"[713, 1075, 1840, 22, 1841, 1076, 7, 27, 0, 0,...","[541, 542, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[4403, 4404, 4405, 4406, 4407, 4408, 4409, 797..."
...,...,...,...
1757,"[102, 138, 2, 74, 279, 16, 56, 0, 0, 0, 0, 0, ...","[126, 127, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[182, 399, 1838, 2214, 718, 1362, 1844, 1, 9, ..."
1758,"[4955, 4956, 4957, 937, 39, 1815, 17, 288, 788...","[244, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1705, 1706, 1707, 1708, 1709, 464, 22, 1, 14,..."
1759,"[179, 147, 611, 4958, 5, 2, 38, 579, 0, 0, 0, ...","[179, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[305, 1, 1268, 949, 4347, 0, 0, 0, 0, 0, 0, 0,..."
1760,"[2, 446, 6, 264, 381, 186, 4959, 67, 573, 4960...","[2, 9, 4, 13, 7, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[156, 157, 4129, 450, 2899, 1, 27, 65, 17, 2, ..."


In [123]:
X_test_new_seq

,title,channel_title,tags
0,"[298, 3288, 68, 220, 78, 534, 573, 91, 110, 35...","[206, 207, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[415, 414, 416, 606, 605, 612, 613, 1, 1, 1, 1..."
1,"[493, 469, 56, 164, 682, 0, 0, 0, 0, 0, 0, 0, ...","[81, 211, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1451, 2, 3, 13, 1452, 1453, 1454, 1455, 226, ..."
2,"[14, 24, 1688, 1210, 128, 3, 1, 91, 0, 0, 0, 0...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 210, 167, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
3,"[113, 4, 657, 37, 509, 28, 59, 1477, 1, 0, 0, ...","[373, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1, 1, 622, 7, 12, 6, 13, 62, 1, 1, 473, 389, ..."
4,"[311, 2282, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[152, 996, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1137, 2853, 1, 1, 1, 1, 4782, 0, 0, 0, 0, ..."
...,...,...,...
436,"[1, 2, 1, 6, 710, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0...","[1308, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 20, 1, 3175, 90, 4184, 57, 70, 1, 1289,..."
437,"[4029, 68, 828, 2, 456, 15, 114, 22, 2980, 298...","[254, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[317, 1765, 829, 1766, 188, 435, 366, 3, 30, 2..."
438,"[463, 440, 75, 164, 682, 0, 0, 0, 0, 0, 0, 0, ...","[81, 211, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1451, 2, 3, 13, 1452, 1453, 1454, 1455, 226, ..."
439,"[260, 3030, 10, 14, 1, 0, 0, 0, 0, 0, 0, 0, 0,...","[692, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[2124, 3573, 1, 445, 1, 3, 2, 3483, 85, 48, 0,..."


In [124]:
X_train_seq = X_train_new_seq['title'].apply(lambda x: np.array(x))
# X_train_seq.append(X_train_new_seq['channel_title'].apply(lambda x: np.array(x)))
# X_train_seq.append(X_train_new_seq['tags'].apply(lambda x: np.array(x)))

In [126]:
X_train_seq.to_numpy()

array([array([ 499, 1838,  250,  709,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0])                                                           ,
       array([1073,   20,  710,   60,   48,   26,   49,   83,    0,    0,    0,
                 0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                 0,    0,    0,    0,  

In [106]:
X_test_new_seq.loc[1,'title'].

AttributeError: 'list' object has no attribute 'shape'

In [84]:
y_train

,category_id
2111,17
518,17
1281,24
805,26
1810,10
...,...
1765,24
1176,24
1214,10
1394,23


In [85]:
y_test

,category_id
834,1
620,22
1887,1
1416,22
1676,26
...,...
663,27
526,23
61,22
73,23


In [88]:
# Define the text model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(units=32))
model.add(Dense(units=1, activation='softmax'))  # Output layer with softmax activation

In [89]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [95]:
X_train_new_seq = np.array(X_train_new_seq)
y_train = np.array(y_train)

In [96]:
X_train_new_seq

array([[array([ 499, 1838,  250,  709,    0,    0,    0,    0,    0,    0,    0,
                  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                  0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
                  0])                                                           ,
        array([540, 322,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
                 0,   0,   0,

In [92]:
y_train

array([[17],
       [17],
       [24],
       ...,
       [10],
       [23],
       [26]])

In [94]:
# Train the model
model.fit(X_train_new_seq, y_train, batch_size=32, epochs=10, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
# Evaluate the model
# Prepare your test data (X_test_text, y_test) in a similar way as the training data
X_test_text_seq = tokenizer.texts_to_sequences(X_test)
X_test_text_padded = pad_sequences(X_test_text_seq, maxlen=max_sequence_length)
loss, accuracy = model.evaluate(X_test_text_padded, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

In [107]:
L = [1,2,3,4,5,6,7,8,9,10]
L

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [108]:
L= np.array(L)

In [109]:
L

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [110]:
print(L)

[ 1  2  3  4  5  6  7  8  9 10]
